In [1]:
 #Import the dataset with mongoimport --type csv -d project_two -c nchs --headerline --drop NCHS_-_Death_rates_and_life_expectancy_at_birth.csv

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# assign the met database to a variable name
db = mongo['project_two']


In [4]:
# review the collections in our new database
print(db.list_collection_names())

['nchs', 'npao']


In [5]:
# assign the collection to a variable
nchs = db['nchs']

In [6]:
# Retrieve and print the data
for document in nchs.find():
    print(document)

{'_id': ObjectId('64f16338a441037430f7b1bf'), 'Year': 1902, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average Life Expectancy (Years)': 51.5, 'Age-adjusted Death Rate': 2301.3}
{'_id': ObjectId('64f16338a441037430f7b1c0'), 'Year': 1900, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average Life Expectancy (Years)': 47.3, 'Age-adjusted Death Rate': 2518.0}
{'_id': ObjectId('64f16338a441037430f7b1c1'), 'Year': 1908, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average Life Expectancy (Years)': 51.1, 'Age-adjusted Death Rate': 2298.9}
{'_id': ObjectId('64f16338a441037430f7b1c2'), 'Year': 1910, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average Life Expectancy (Years)': 50.0, 'Age-adjusted Death Rate': 2317.2}
{'_id': ObjectId('64f16338a441037430f7b1c3'), 'Year': 1909, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average Life Expectancy (Years)': 52.1, 'Age-adjusted Death Rate': 2249.2}
{'_id': ObjectId('64f16338a441037430f7b1c4'), 'Year': 1911, 'Race': 'All Races', 'Sex': 'Both Sexes', 'Average

In [7]:
sample_document = nchs.find_one()
sample_document

{'_id': ObjectId('64f16338a441037430f7b1bf'),
 'Year': 1902,
 'Race': 'All Races',
 'Sex': 'Both Sexes',
 'Average Life Expectancy (Years)': 51.5,
 'Age-adjusted Death Rate': 2301.3}

In [8]:
pipeline = [
    {
        "$match": {
            "Race": "All Races"
        }
    },
    {
        "$group": {
            "_id": "$Year",
            "max_life_expectancy": {"$max": "$Average Life Expectancy (Years)"}
        }
    },
    {
        "$sort": {
            "max_life_expectancy": -1  # Sort in descending order (highest first)
        }
    },
    {
        "$limit": 1
    }
]

# Execute the aggregation pipeline
result = list(nchs.aggregate(pipeline))

# Check if any result was found
if result:
    highest_life_expectancy_year = result[0]["_id"]
    highest_life_expectancy_value = result[0]["max_life_expectancy"]
    print(f"The year with the highest life expectancy for 'All Races' is {highest_life_expectancy_year} with an average life expectancy of {highest_life_expectancy_value} years.")

The year with the highest life expectancy for 'All Races' is 2014 with an average life expectancy of 81.3 years.


In [9]:
# Define the field to c81.3 years.heck for empty values
field_to_check = "Average Life Expectancy (Years)"

# Define the aggregation pipeline to filter out documents with empty data
pipeline = [
    {
        "$match": {
            field_to_check: {"$exists": True, "$ne": None, "$ne": ""}
        }
    },
    {
        "$match": {
            "Race": "White"
        }
    },
    {
        "$group": {
            "_id": "$Year",
            "max_life_expectancy": {"$max": "$Average Life Expectancy (Years)"}
        }
    },
    {
        "$sort": {
            "max_life_expectancy": -1  # Sort in descending order (highest first)
        }
    },
    {
        "$limit": 1
    }
]

# Execute the aggregation pipeline
result = list(nchs.aggregate(pipeline))

# Check if any result was found
if result:
    highest_life_expectancy_year = result[0]["_id"]
    highest_life_expectancy_value = result[0]["max_life_expectancy"]
    print(f"The year with the highest life expectancy for 'White' is {highest_life_expectancy_year} with an average life expectancy of {highest_life_expectancy_value} years.")

The year with the highest life expectancy for 'White' is 2013 with an average life expectancy of 81.4 years.


In [15]:
# Define the aggregation pipeline to filter out documents with empty data
pipeline = [
    {
        "$match": {
            field_to_check: {"$exists": True, "$ne": None, "$ne": ""}
        }
    },
    {
        "$match": {
            "Race": "Black"
        }
    },
    {
        "$group": {
            "_id": "$Year",
            "max_life_expectancy": {"$max": "$Average Life Expectancy (Years)"}
        }
    },
    {
        "$sort": {
            "max_life_expectancy": -1  # Sort in descending order (highest first)
        }
    },
    {
        "$limit": 1
    }
]

# Execute the aggregation pipeline
result = list(nchs.aggregate(pipeline))

# Check if any result was found
if result:
    highest_life_expectancy_year = result[0]["_id"]
    highest_life_expectancy_value = result[0]["max_life_expectancy"]
    print(f"The year with the highest life expectancy for 'Black' is {highest_life_expectancy_year} with an average life expectancy of {highest_life_expectancy_value} years.")

The year with the highest life expectancy for 'Black' is 2017 with an average life expectancy of 78.5 years.


In [14]:

#NO SURE IF WE NEED THIS!!!
# Convert the 'result' list to a DataFrame
df = pd.DataFrame(result)

# Calculate summary statistics
summary_stats = df.groupby('_id')['count'].agg(['mean', 'median', 'std', 'min', 'max']).reset_index()

# Rename columns for clarity
summary_stats.columns = ['Year', 'Mean', 'Median', 'Std', 'Min', 'Max']

# Print the summary statistics
print(summary_stats)

KeyError: 'Column not found: count'

In [23]:
# Query
query = {'Year': {'$gte': 2010}}
fields = {'Race': 1, 'Year': 1, 'Sex': 1, 'Average Life Expectancy (Years)': 1, 'Age-adjusted Death Rate': 1}
sort = [('Year', -1)]
#limit = 5

# Cast the results as a list and save them to a variable
results = list(nchs.find(query, fields).sort(sort))
#results = list(nchs.find(query, fields).sort(sort).limit(limit))

# Pretty print the results
pprint(results)

[{'Age-adjusted Death Rate': 723.6,
  'Average Life Expectancy (Years)': 78.7,
  'Race': 'All Races',
  'Sex': 'Both Sexes',
  'Year': 2018,
  '_id': ObjectId('64f16338a441037430f7b234')},
 {'Age-adjusted Death Rate': 611.3,
  'Average Life Expectancy (Years)': 81.2,
  'Race': 'All Races',
  'Sex': 'Female',
  'Year': 2018,
  '_id': ObjectId('64f16338a441037430f7b2b5')},
 {'Age-adjusted Death Rate': 855.5,
  'Average Life Expectancy (Years)': 76.2,
  'Race': 'All Races',
  'Sex': 'Male',
  'Year': 2018,
  '_id': ObjectId('64f16338a441037430f7b31d')},
 {'Age-adjusted Death Rate': 852.9,
  'Average Life Expectancy (Years)': '',
  'Race': 'Black',
  'Sex': 'Both Sexes',
  'Year': 2018,
  '_id': ObjectId('64f16338a441037430f7b399')},
 {'Age-adjusted Death Rate': 702.6,
  'Average Life Expectancy (Years)': '',
  'Race': 'Black',
  'Sex': 'Female',
  'Year': 2018,
  '_id': ObjectId('64f16338a441037430f7b432')},
 {'Age-adjusted Death Rate': 1051.5,
  'Average Life Expectancy (Years)': '',
  '

In [25]:
aggregated_df = pd.json_normalize(results)
aggregated_df

,_id,Year,Race,Sex,Average Life Expectancy (Years),Age-adjusted Death Rate
0,64f16338a441037430f7b234,2018,All Races,Both Sexes,78.7,723.6
1,64f16338a441037430f7b2b5,2018,All Races,Female,81.2,611.3
2,64f16338a441037430f7b31d,2018,All Races,Male,76.2,855.5
3,64f16338a441037430f7b399,2018,Black,Both Sexes,,852.9
4,64f16338a441037430f7b432,2018,Black,Female,,702.6
...,...,...,...,...,...,...
76,64f16338a441037430f7b403,2010,Black,Female,78.0,752.5
77,64f16338a441037430f7b479,2010,Black,Male,71.8,1104.0
78,64f16338a441037430f7b4f1,2010,White,Both Sexes,78.9,741.8
79,64f16338a441037430f7b56e,2010,White,Female,81.3,630.8
